<a href="https://colab.research.google.com/github/KANG-dg/porject/blob/main/%EC%9E%8E%EB%B6%84%EB%A5%98_transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import torch.nn as nn  # 딥러닝 네트워크 구성요소
import torch.nn.functional as F # 딥러닝 함수 포함 모듈
import torch.optim as optim # 가중치 추적에 필요한 최적화 알고리즘

from tqdm.auto import tqdm

In [ ]:
import torch
USE_CUDA=torch.cuda.is_available()              # 현재 환경에서 GPU를 사용할 수 있는지
DEVICE=torch.device('cuda'if USE_CUDA else 'cpu') # 사용하는 장비가 무엇인지 저장하는 변수 앞의 코드에서 gpu사용 가능 여부가 True면 'cuda'를 false 면 'cpu' 장비를 변수에 저장

BATCH_SIZE=256       # 배치 사이즈 256으로 설정 모델 학습,평가 과정에서 배치 사이즈 여러번 입력해야 하는데 배치사이즈를 한번에 모두 바꾸기 위해 변수만들어서 설정
EPOCH=30            # epoch는 30 변수 생성 이유는 위와 같음

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base=transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])    # transforms.compose()는 이미지 데이터 전처리, augmentation등 과정에서 사용되는 메소드 augmentainon의 종류로는 좌우반전, 밝기 조절, 이미지 임의 확대 등 존재
                                                                                        # transforms.Resize(64,64)는 이미지 크기를 64*64로 조정 transforms.ToTensor()는 이미지를 tensor형태로 변환하고 모든 값을 0에서 1사이로 정규화
train_dataset=ImageFolder(root='/content/drive/MyDrive/torch_data/splitted/train', transform=transform_base)    # imagefolder메서드는 데이터셋을 불러옴 여기서 이미지데이터는 하나의 클래스가 하나의 폴더에 대응 이러한 구조에서 imagefolder를 사용
                                                                                                          # root옵션에 데이터 불러올 경로를 설정 transform옵션에는 데이터를 불러온 후 전처리 혹은 augmentation할 방법을 지정 transform_base로 옵션 설정
val_dataset=ImageFolder(root='/content/drive/MyDrive/torch_data/splitted/val', transform=transform_base) # 검증을 위한 데이터를 val폴더에 접근해 학습데이터와 동일하게 전처리 되도록 설정

from torch.utils.data import DataLoader

train_loader=torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) # dataloader는 불러온 이미지 데이터를 주어진 조건에 따라 미니 배치 단위로 분리하는 역할을 수행함 학습과정에 dataloader는 train_dataset을 
                                                                                                            # 이용하여 생성 shuffle=True 로 성정하면 데이터의 순서가 섞여 모델이 학습할 때 label정보의 순서를 기억하는 것을 방지

val_loader=torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) # 검증 과정에 사용될 Dataloader는 imageFolder인 val_dataset을 이용하여 생성 다른 조건은 train_folder와 동일하게 설정

In [ ]:
data_transforms={
    'train': transforms.Compose([ # transforms.compose는 이미지 데이터의 전처리 augmentation 과정에 사용되는 메서드
    transforms.Resize([64,64]),  # 이미지 크기 64*64로 조정
    transforms.RandomHorizontalFlip(), # 좌우반전
    transforms.RandomVerticalFlip(), # 상하반전
    transforms.RandomCrop(52), # 이미지 일부를 랜덤하게 잘라내어 52*52로 변경 
    transforms.ToTensor(),  # 이미지를 tensor형태로 변환 모둔 숫자응 0에서 1사이로
    transforms.Normalize([0.485,0.456,0.406], # 이미지 tensor로 변환후 정규화 시행 정규화를 위해 평균과 표준편차필요 Normalize() 메서드의 첫번째 대괄호는 RGB채널 값에서 정규화를 적용할 평균값 의미
                         [0.229,0.224,0.225]) # 두 번째 대괄호는 각 RGB채널 값에서 정규화를 적용할 표준편차값 의미 사용된 평균, 표준편차는 pre_trained_model학습에 사용된 imagenet 데이터의 값임  
    ]),                                       # 입력 데이터 정규화는 모델을 최적화하고 local minimum에 빠지는 것을 방지하는데 도움이 됨
    'val': transforms.Compose([             
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) # train 의 augmentation에 해당하는 부분을 제외하고 동일하게 사용
    ])}


data_dir='/content/drive/MyDrive/torch_data/splitted' # 학습데이터와 검증 데이터를 불러올 경로 설정
image_datasets={x:ImageFolder(root=os.path.join(data_dir,x),  # ImageFolder 메서드는 데이터셋을 불러오는 메서드 root옵션에 데이터를 불러올 경로를 설정
                              transform=data_transforms[x])for x in ['train','val']} # transform 옵션에 데이터를 불러온 후 전처리 또는 augmentation 방법을 지정(앞서 정의한 transform_base로 옵션 지정) 훈련, 검증에 맞는 데이터 쉽게 불러오기위해 dict타입으로
dataloaders={x:torch.utils.data.DataLoader(image_datasets[x],                                  # DataLoader는 불러온 이미지 데이터를 조건에 따라 미니 배치 단위로 분리하는 역할 수행 위에서 설정한 image_dataset을 통해 생성
                                           batch_size=BATCH_SIZE, shuffle=True, num_workers=4)  # shuffle=True로 설정하면 데이터의 순서가 섞여 모델이 학습할 때 label정보의 순서를 기억하는 것 방지가능 마찬가지로 dict타입으로 구성
                                            for x in ['train','val']}

dataset_sizes={x:len(image_datasets[x]) for x in ['train','val']}  # 이후활용을 위해 학습데이터와 검증데이터의 총 개수를 각각 저장합니다 마찬가지로 dict타입으로 구성 

class_names=image_datasets['train'].classes # 이후 활용을 위해 33개 클래스 이름목록 저장



In [ ]:
#   참고_____convnext같은 경우 이렇게 하면 됨!!!!
con=models.convnext_base(pretrained=True)
num_con=con.classifier[2].in_features
num_con

1024

In [ ]:
from torchvision import models
resnet=models.resnet50(pretrained=True)  # resnet50모델 사용 pretrained=True로 하면 미리 학습된 모델의 parameter값을 그대로 가지고옴 false로 하면 모델의 구조만을 가져오고 parameter값은 랜덤
num_ftrs=resnet.fc.in_features  # 데이터를 33개 클래스로 분류해야하기에 모델 마지막 출력레이어는 33개여야함 하지만 불러온 resnet50모델은 다른 주제를 위해 설계되었기에 마지막레이어 33개 아님
                                # 따라서 모델의 마지막 fully connected layer 대신 출력 채널의 수가 33개인 새로운 layer추가 할 것 이를위해 불러온 resnet50에서 마지막 layer의 입력 채널수를 저장 in_features는 해당 layer의 입력채널 수 의미
resnet.fc=nn.Linear(num_ftrs,33) # 모델의 마지막 fully connected layer를 새로운 layer로 교체 이때 입력 채널수는 기존 layer와 동일하고 출력 채널수는 33개로 설정
resnet=resnet.to(DEVICE) # to.(DEVIE) 메스드를 통해 모델을 장비에 할다

criterion=nn.CrossEntropyLoss() # 모델 학습시에 사용하는 loss함수를 지정하는 변수 cross entropy loss 를 사용

optimizer_ft=optim.Adam(filter(lambda p:p.requires_grad, resnet.parameters()), lr=0.001) # optimizer 는 adam으로 learning late 는 0.001로 설정 베이스라인 모델에서는 모든 parameter를 업데이트 했지만 이 모델에서는 일부 layer의 parameter만 업데이트
                                                                                        # 따라서 filter 메서드와 lambda 표현식을 사용하여 requires_grad=True로 설정된 layer의 parameter만 적용 
                                                                                        # filter() 함수는 두번째 인자로 넘어온 데이터 중에서 첫번째 인자로 넘어온 조건 함수를 만족하는 데이터만을 반환합니다.

from torch.optim import lr_scheduler

exp_lr_scheduler=lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) # stepLR메서드는 epoch에 따라 learning rate를 변경하는 역할을 한다 step_size=7, gamma=0.1로 설정하면 7epoch마다 0.1씩 곱해 learning rate를 감소시킨다는 의미

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
# pretrained model 의 일부 layer freeze 하기

ct=0  # 해당 layer가 몇번째 layer인지 나타내는 변수 ct의 값을 0으로 초기화
for child in resnet.children():   # children() 메서드는 모델의 자식 모듈을 반복 가능한 객체로 반환하는 메서드  resnet.childeren() 은 생성한 resnet모델의 모든 layer정보를 담고 있음
  ct+=1    # for문을 한번 반복한 후에는 다음 layer를 지칭하도록 변수 ct의 값을 1만큼 증가시킴
  if ct<6:   # 미리 학습된 모델의 일부 layer만 업데이트하도록 설정하는 방법은 parameter를 업데이트하지 않을 상위 layer의 requires_grad값을 false로 지정하는 것
    for param in child.parameters():   # resnet에 존재하는 10개의 layer중에서 1번부터 5번까지 layer의 parametr는 업데이트 되지 않도록 고정하고 6번부터 10번layer의 parameter는 학습과정에서 업데이트 하도록 설정
      param.requires_grad=False        # child.parameters() 는 각 layer의 parametr tensor를 의미함 각 텐서에는 requires_grad 옵션이 있고 기본값은 True로 지정되어 있음
                                      # requires_grad=False는 parameter가 업데이트되지 않도록 설정한다는 의미 layer의 번호가 6보다 잗은 경우 parameter가 업데이트 되지 않도록 설정

In [ ]:
dataloaders['train']

In [ ]:
import time
import copy

def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):
  best_model_wts=copy.deepcopy(model.state_dict())  # 정확도가 가장 높은 모델을 저장할 변수 best_model_wts를 선언
  best_acc=0.0  # 정확도가 가장 높은 모델의 정확도를 저장하는 변수 best_acc를 선언하고 값을 0으로 초기화

  for epoch in range(num_epochs):
    print('-------------epoch{}---------------'.format(epoch+1))  # 현재 진행중인 epoch 출력
    since=time.time()   # epoch당 소요되는 시간을 측정하기위해 epoch 시작할 때 시각을 저장

    for phase in ['train','val']:  # 한 epoch은 각각 학습단계와 검증단계를 가짐 for문으로 한 epoch마다 학습모드와 검증모드 각각 실행
      if phase=='train':
        model.train() # 학습모드 설정
      else:
        model.eval() # 검증모드 설정

      running_loss=0.0  # 모든 데이터의 loss를 합산하여 저장할 변수인 running loss를 선언하고 0으로 초기화
      running_corrects=0 # 올바르게 예측한 경우의 수를 세는 변수인 running_corrects를 선언하고 0으로 초기화

      for inputs, labels in tqdm(dataloaders[phase]):  # 모델의 현재모드에 해당하는 dataloader에서 데이터를 입력받음
        inputs=inputs.to(DEVICE) # 데이터와 해당하는 label을 현재 사용중인 장비에 할당
        labels=labels.to(DEVICE)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase=='train'):  # 학습단계에만 모델의 gradient를 업데이트 하고 검증단게에서는 업데이트 하지 않아야함 set_grad_enabled() 메서드를 이용해 phase가 train일때만 업데이트 함
          outputs=model(inputs) # 데이터를 모델에 입력하여 outputs 값 계산ㄴ
          _,preds=torch.max(outputs,1) # 모델에 입력된 test데이터가 33개의 클래스에 속할 각각의 확률값이 outut으로 출력됨 이 중 가장 높은 값을 가진 인덱스를 예측값으로 저장
          loss=criterion(outputs, labels) # 모델에서 계산한 output값인 예측값과 target값 사이의 loss를 계산함 입력바는 loss함수를 통해 loss계산

          if phase=='train': # 모델이 학습모드일 경우 위에서 계산한 loss를 바탕으로 back propagation을 통해 계산된 gradient값을 각 parameter에 할당하고 모델의 parameter를 업데이트 
            loss.backward()
            optimizer.step()
        
        running_loss+=loss.item()*inputs.size(0) # 모든 데이터loss를 합산해 저장하고자 하나의 미니 배치에 대해 계산된 loss의 값에 데이터의 수를 곱해 합산 이 때 input.size(0)은 dataloader에서 전달되는 미니 배치의 데이터수를 의미==배치사이즈
        running_corrects+=torch.sum(preds==labels.data) # 위에서 모델을 통해 예측한 값과 target이 같으면 running_corrects를 1만큼 증가 같지 않으면 증가시키지 않음
      
      if phase=='train':  # 7epoch마다 learning rate를 다르게 저정하는 scheduler와 관련된 부분 한 epoch당 1번 모델이 학습단계일 경우만 실행
        scheduler.step()
        l_r=[x['lr']for x in optimizer_ft.param_groups] # scheduler에 의해 learning rate가 조정되는 것을 직접 확인하기위한 부분 optimizer_ft.param_groups의 원소는 학습과정의 parameter를 저장하고 있는 딕셔너리
        print('learning rate:', l_r)                    # 이 중 learing rate에 하댕하는 key인 lr을 이용하여 epoch의 learning rate를 불러옴 
      
      

      epoch_loss=running_loss/dataset_sizes[phase]  # 해당 epoch의 loss를 계산하기위해 running_loss를 미리 계산해둔 dataset_size로 나눔
      epoch_acc=running_corrects.double()/dataset_sizes[phase] # 해당 epoch의 정확도를 계산라기위해 running_correct를 미리 계산해둔 dataset_size로 나눔

      print('{}Loss:{:.4f} Acc{:.4f}'.format(phase, epoch_loss, epoch_acc)) # 해당 epoch과 현재 모델의 단계, loss값, 정확도를 출력

      if phase=='val' and epoch_acc>best_acc:  # 검증단계에서 현재 epoch의 정확도가 최고 정확도보다 높다면 best_acc를 현재 epoch의 정확도로 업데이트하고 해당 epoch의 모델을 best_model_wts에 저장
        best_acc=epoch_acc
        best_model_wts=copy.deepcopy(model.state_dict())

    

    time_elapsed=time.time()-since  # 한 epoch당 소요된 시간으 ㄹ계산 해당 epoch을 시작할 떄 시간을 epoch이 끝날 때의 시각에서 뺀 값 ㄱ쎄나
    print('completed in {:.0f}m {:.0f}s'. format(time_elapsed//60, time_elapsed%60))
  print('best val acc:{:4f}'.format(best_acc))

  model.load_state_dict(best_model_wts) # 정확도가 가장 높은 모델을 불러온후 반환

  return model



In [ ]:
def evaluate(model, test_loader):
  model.eval()  # 입력받는 모델 평가모드로
  test_loss=0 # 미니배치별로 loss를 합산해서 저장할 변수 test_loss를 선언하고 0으로 초기화
  correct=0 # 올바르게 예측한 데이터의 수를 세는 변수인 correct를 선언하고 0으로 초기화

  with torch.no_grad():  # 모델 평가단계에서는 모델 parametr를 업데이트 하지 않아야 함 with torch.no_grad를 통해 해당부분 실행하는 동안 모델의 parameter업데이트 중단
    for data, target in tqdm(test_loader):  # test_load에는 data, targe이 미니배치 단위로 묶여 있음 for문을 통해 데이터와 대응하는 label에 접근
      data, target=data.to(DEVICE), target.to(DEVICE)  # 장비로 할당
      output=model(data)  # 데이터를 모델에 입력하여 output을 계산


      test_loss+=F.cross_entropy(output, target, reduction='sum').item() # 모델에서 계산한 output인 예측값과 target 값 사이의 loss 를 계산 cross entropy loss 함수를 사용

      pred=output.max(1, keepdim=True)[1]  # 모델에 입력된 test데이터가 33개 클래스에 속할 각 확률값이 output으로 출력 이 중 가장 높은 값을 가진 인덱스를 예측값으로 저장
      correct+=pred.eq(target.view_as(pred)).sum().item() # target.view_as(pred)를 통해 target tensor 구조를 pred tensor와 같은 모양으로 정렬 view_as() 메서드는 적용대상 tensor를 메서드에 입력되는 tensor의 모양대로 재정렬하는 함수
                                                          # 모델 구조를 구성할 때 학습했던 view() 함수는 정렬하고 싶은 tensor의 모양을 숫자로 직접 지정한다는 점에서 차이 있음 
                                                          # eq 메서드는 객체간 비교 연산자로 pred와 target.view_as(pred)의 값이 일치하면 1 아니면 0을 반환

  test_loss/=len(test_loader.dataset) # 모든 미니배치에서 합한 loss값을 batch수로 나누어 미니 배치마다 계산도니 loss의 값이 평균을 구함
  test_accuracy=100.*correct/len(test_loader.dataset) # 모든 미니배치에서 합한 정확도 값을 batch 수로 나누어 미니 배치마다 계산된 정확도의 평군을 구함
  return test_loss, test_accuracy # 측정한 test loss와 정확도 반환

In [ ]:
model_resnet50=train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH)  # 앞서 정의한 train_resnet합수를 이용해 resnet50모델을 fine_tuning 함
torch.save(model_resnet50, '/content/drive/MyDrive/torch_data/resnet50.pt') # 학습이 완료된 모델 model_resnet50을 resnet50.pt 이름으로 저장


-------------epoch1---------------


  0%|          | 0/94 [00:00<?, ?it/s]

learning rate: [0.001]
trainLoss:0.5896 Acc0.8248


  0%|          | 0/32 [00:00<?, ?it/s]

valLoss:0.3267 Acc0.8987
completed in 27m 28s
-------------epoch2---------------


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd5cdd96430>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fd5cdd96430>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
        self._shutdown_workers()self._shutdown_workers()

  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
        if w.is_alive():if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive

Exception ignored in:   File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    <function _MultiProcessingDataLoaderIter

learning rate: [1.0000000000000002e-07]
trainLoss:0.0116 Acc0.9964


  0%|          | 0/32 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd5cdd96430>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd5cdd96430>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

valLoss:0.0324 Acc0.9905
completed in 0m 54s
best val acc:0.990737


In [ ]:
# 모델평가

transform_resNet=transforms.Compose([
    transforms.Resize([64,64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_resNet=ImageFolder(root='/content/drive/MyDrive/torch_data/splitted/test', transform=transform_resNet)

test_loader_resNet=torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
resnet50=torch.load('/content/drive/MyDrive/torch_data/resnet50.pt')
resnet50.eval()
test_loss, test_accuracy=evaluate(resnet50, test_loader_resNet)

  0%|          | 0/32 [00:00<?, ?it/s]